# RNN from scratch: a character-level language model
I am going to train this RNN on a datasets of names, then sample from it to see if it can generate some nanem-like texts.

### Data I/O and initialise the model

In [10]:
from RNN import RNN
import numpy as np

# load training data
with open('names.txt', 'r') as f:
    data = f.read() # chars

# data I/O
chars = sorted(list(set(data)))
data_size = len(data)
vocab_size = len(chars)
stoi = {j: i for i, j in enumerate(chars)} # mapping from char to index
itos = {i: j for i, j in enumerate(chars)} # mapping from index to char
print(f'Data has {data_size} characters, {vocab_size} unique.')

# define the model 
model = RNN(300, 50, vocab_size, data_size, stoi, itos)

Data has 228145 characters, 27 unique.


### Generate some results before training.

In [11]:
model.generate(200)

ohbomstlpenl
sr
znd
chyzilakzavomgdrhjpmfobbuuqmvvabvvwfbwxa
eaatvflfmxmetbzlwkwblaawsbywlzcycwgwscmnthyvhdncz
lwrwvvrhxfjtfnpxxsxdsuugwrxkrorehrrmsjidvwirvspcmzgmwpkhczzf
lsoqntoyqzwygjkk
tbkrqgmpfvp


Yup, rubbish...

### Now train the model 

In [12]:
model.train(data, total_iteration=10000, lr=1e-5)

Iteration : 0 | Loss : 164.7696103142331
Iteration : 10 | Loss : 164.66535718496777
Iteration : 20 | Loss : 164.51701583676132
Iteration : 30 | Loss : 164.32600885308727
Iteration : 40 | Loss : 163.49846303382793
Iteration : 50 | Loss : 157.38773949080527
Iteration : 60 | Loss : 141.05223536913616
Iteration : 70 | Loss : 131.1420115276438
Iteration : 80 | Loss : 143.9113589641888
Iteration : 90 | Loss : 132.94304767362473
Iteration : 100 | Loss : 139.28577424410642
Iteration : 110 | Loss : 140.33596795618132
Iteration : 120 | Loss : 144.80674511743516
Iteration : 130 | Loss : 134.33710421304332


KeyboardInterrupt: 